# SQPLM

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.5.20


In [2]:
model = uf.SQPLM("../../ref/bert_config.json", "../../ref/vocab.txt", do_sample_next_sentence=False)
print(model)

INFO:tensorflow:LM Mode: `bi`. Use method `.to_mode()` to convert it into `bi`, `l2r`, `r2l` or `s2s`.
uf.SQPLM(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=128,
    label_size=None,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    drop_pooler=False,
    do_sample_next_sentence=False,
    max_predictions_per_seq=20,
    masked_lm_prob=0.15,
    short_seq_prob=0.1,
    do_whole_word_mask=False,
    mode="bi",
    do_lower_case=True,
    truncate_method="LIFO",
)


可通过简单的一项指令，在四种训练模式中切换

In [3]:
model.to_mode("bi")    # bi/l2r/r2l/s2s

In [4]:
# bi/l2r/r2l模式下
X = [
     ["天亮以前说再见，", "笑着泪流满面，", "去迎接应该你的，", "更好的明天。"],     # 每一条样本是一个doc，doc内可以由多个句子组成
     "他想知道那是谁, 为何总沉默寡言, 人群中也算抢眼, 抢眼的孤独难免",               # 也可以是一个完整的文段
]

# s2s模式下
# X = [["天亮以前说再见", "笑着泪流满面"], ["他想知道那是谁", "为何总沉默寡言"]]     # doc必须为两个句子

# 训练

In [5]:
model.fit(X, [0] * len(X), total_steps=20)

/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:246: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:256: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  value_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.20-py3.8.egg/uf/apps/bert/bert.py:380: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Ple

INFO:tensorflow:Build graph with 308,647,326 parameters (among which 102,882,442 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 2 samples (step 0 -> 20)
INFO:tensorflow:step 1, MLM accuracy 0.0000, MLM loss 2.566339, NSP accuracy 0.0000, NSP loss 0.854683, 0.13 steps/sec, 0.26 examples/sec
INFO:tensorflow:step 2, MLM accuracy 0.1000, MLM loss 2.195817, NSP accuracy 1.0000, NSP loss 0.119030, 0.48 steps/sec, 0.95 examples/sec
INFO:tensorflow:step 3, MLM accuracy 0.8000, MLM loss 1.855942, NSP accuracy 1.0000, NSP loss 0.038016, 0.52 steps/sec, 1.03 examples/sec
INFO:tensorflow:step 4, MLM accuracy 0.9000, MLM loss 1.634639, NSP accuracy 1.0000, NSP loss 0.022335, 0.52 steps/sec, 1.04 examples/sec
INFO:tensorflow:step 5, MLM accuracy 0.8000, MLM loss 1.454528, NSP accuracy 1.0000, NSP loss 0.018225, 0.53 steps/sec, 1.05 examples/sec
INFO:tensorflow:step 6, MLM accuracy 1.0000, MLM loss 1.301332, NSP accu

# 推理

In [6]:
X_tokenized = [
    [
        token for token 
        in model.tokenizer.convert_ids_to_tokens(model.data["input_ids"][i])
        if token not in ("[CLS]", "[SEP]", "[PAD]")
    ]
    for i in range(len(model.data["input_ids"]))
]
print(X_tokenized)
model.predict(X_tokenized=X_tokenized)

[['天', '亮', '以', '前', '说', '再', '见', '，', '笑', '[MASK]', '泪', '流', '满', '面', '，', '[MASK]', '迎', '接', '应', '该', '你', '[MASK]', '[MASK]', '更', '好', '的', '明', '天', '。'], ['他', '想', '知', '[MASK]', '那', '是', '谁', ',', '[MASK]', '何', '总', '沉', '默', '寡', '言', ',', '人', '群', '[MASK]', '也', '算', '抢', '眼', ',', '[MASK]', '眼', '的', '[MASK]', '独', '难', '免']]
INFO:tensorflow:Running inference on 2 samples
INFO:tensorflow:process 100.0%, 1.56 examples/sec


{'mlm_preds': [['的', '去', '的', '你'], ['道', '为', '中', '抢', '孤']],
 'nsp_preds': [0, 0],
 'nsp_probs': array([[0.99808383, 0.00191617],
        [0.99836093, 0.00163909]], dtype=float32)}

# 训练样本生成

In [7]:
X = ["天亮以前说再见", "笑着泪流满面", "去迎接应该你的", "更好的明天", "他想知道那是谁", "为何总沉默寡言", "人群中也算抢眼", "抢眼的孤独难免"]
data = model.convert(X, [0] * len(X), is_training=True)
for i in range(len(data["input_ids"])):
    print("input_tokens: %s" % (model.tokenizer.convert_ids_to_tokens(data["input_ids"][i])))
    print("")


input_tokens: ['[CLS]', '天', '亮', '以', '前', '说', '[MASK]', '见', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 